In [ ]:
!pip install dash

In [1]:
import dash
from dash import dcc, html, Input, Output, State
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import joblib
import os
import time
import json


## Load all models

In [2]:
models = {}
try:
    models['FIFA'] = joblib.load('fifa_model.joblib')
    print("FIFA model loaded successfully!")
except Exception as e:
    print(f"FIFA model loading error: {e}")
    models['FIFA'] = None

try:
    models['NBA'] = joblib.load('nba_model.joblib')
    print("NBA model loaded successfully!")
except Exception as e:
    print(f"NBA model loading error: {e}")
    models['NBA'] = None

try:
    models['NFL'] = joblib.load('nfl_model.joblib')
    print("NFL model loaded successfully!")
except Exception as e:
    print(f"NFL model loading error: {e}")
    models['NFL'] = None

FIFA model loaded successfully!
NBA model loaded successfully!
NFL model loaded successfully!


## Function

In [3]:
def create_shared_metrics():
    return html.Div([
        html.H3("Shared Athlete Metrics", style={'marginBottom': '20px', 'textAlign': 'center'}),
        
        html.Div([
            # Left column - Height and Weight
            html.Div([
                html.Label("Height (cm)"),
                dcc.Slider(
                    id='height-slider', 
                    min=150, 
                    max=230, 
                    step=1, 
                    value=185, 
                    marks={i: str(i) for i in range(150, 231, 10)},
                    tooltip={"placement": "bottom", "always_visible": True}
                ),
                
                html.Label("Weight (kg)", style={'marginTop': '15px'}),
                dcc.Slider(
                    id='weight-slider', 
                    min=50, 
                    max=150, 
                    step=1, 
                    value=80, 
                    marks={i: str(i) for i in range(50, 151, 10)},
                    tooltip={"placement": "bottom", "always_visible": True}
                ),
            ], className='six columns'),
            
            # Right column - Age and BMI display
            html.Div([
                html.Label("Age"),
                dcc.Slider(
                    id='age-slider', 
                    min=16, 
                    max=40, 
                    step=1, 
                    value=25, 
                    marks={i: str(i) for i in range(16, 41, 4)},
                    tooltip={"placement": "bottom", "always_visible": True}
                ),
                
                # Automatically calculated BMI
                html.Div([
                    html.Label("BMI (automatically calculated):", 
                              style={'display': 'inline-block', 'marginRight': '10px', 'marginTop': '15px'}),
                    html.Div(
                        id='bmi-display', 
                        style={'display': 'inline-block', 'fontWeight': 'bold', 'marginTop': '15px'}
                    )
                ]),
            ], className='six columns'),
        ], className='row'),
        
        html.Hr(style={'marginTop': '20px', 'marginBottom': '20px'}),
    ], style={'padding': '20px', 'backgroundColor': '#FFFFFF', 'borderRadius': '5px', 'boxShadow': '0px 0px 5px lightgrey', 'marginBottom': '20px'})


In [4]:
def get_input_data(league, height, weight, age, bmi, position=None, years_pro=None, enter_pro=None):
    input_data = {}
    
    if league == 'FIFA':
        # FIFA model features
        input_data = {
            'height_cm': height,
            'weight_kg': weight,
            'Age': age,
            'bmi': bmi
        }
    else:  # NBA or NFL
        # NBA and NFL model features
        input_data = {
            'height_cm': height,
            'weight_kg': weight,
            'position_label': position,
            'Age': age,
            'years_pro': years_pro,
            'bmi': bmi,
            'enter_pro': enter_pro if enter_pro is not None else age - years_pro
        }
    
    return input_data

In [5]:
def predict_future_ratings(league, initial_data, years=10):
    model = models.get(league)
    if model is None:
        raise ValueError(f"No model available for {league}")
    
    predictions = []
    ages = []
    features_list = []
    
    # Deep copy of initial data to avoid modifying the original
    data = dict(initial_data)
    
    for year in range(years):
        # Current age
        current_age = data['Age'] + year
        ages.append(current_age)
        
        # Make a copy of features for this year
        year_features = dict(data)
        
        # Update age and years pro for current prediction year
        if year > 0:
            data['Age'] += 1
            if 'years_pro' in data:
                data['years_pro'] += 1
        
        # Prepare feature values in the correct order for model prediction
        if league == 'FIFA':
            feature_values = [data['height_cm'], data['weight_kg'], data['Age'], data['bmi']]
        else:  # NBA or NFL
            feature_values = [
                data['height_cm'], 
                data['weight_kg'], 
                data['position_label'], 
                data['Age'], 
                data['years_pro'], 
                data['bmi'], 
                data['enter_pro']
            ]
        
        # Store features for this year
        features_list.append(dict(data))
        
        # Predict rating for this year
        try:
            # Handle both dictionary and direct model formats
            if isinstance(model, dict) and 'model' in model:
                prediction = model['model'].predict([feature_values])[0]
            else:
                prediction = model.predict([feature_values])[0]
            
            predictions.append(prediction)
        except Exception as e:
            print(f"Prediction error for year {year}: {e}")
            # If prediction fails, use last valid prediction or default value
            if predictions:
                predictions.append(predictions[-1])
            else:
                predictions.append(70.0)  # Default value if no valid predictions
    
    return predictions, ages, features_list

In [6]:
def create_prediction_results(predictions, ages, league_color):
    # Create a DataFrame for the chart
    df = pd.DataFrame({
        'Age': ages,
        'Overall Rating': [round(p, 2) for p in predictions]
    })
    
    # Find the max rating point
    max_prediction = max(predictions)
    max_index = predictions.index(max_prediction)
    max_age = ages[max_index]
    
    # Create the figure
    fig = px.line(df, x='Age', y='Overall Rating',
                title=f'10-Year Overall Rating Prediction',
                markers=True)
    
    # Highlight the peak rating
    fig.add_trace(go.Scatter(
        x=[max_age], 
        y=[max_prediction],
        mode='markers',
        marker=dict(color='red', size=12),
        name='Peak Rating'
    ))
    
    # Add annotation for the peak
    fig.add_annotation(
        x=max_age,
        y=max_prediction,
        text=f'Peak Rating: {max_prediction:.2f}<br>Age: {max_age}',
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=70,
        ay=-40,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor=league_color,
        opacity=0.8
    )
    
    # Update the layout
    fig.update_layout(
        xaxis_title="Age",
        yaxis_title="Overall Rating",
        hovermode="closest",
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis=dict(gridcolor='lightgrey'),
        yaxis=dict(gridcolor='lightgrey'),
        margin=dict(l=20, r=20, t=40, b=20),
    )
    
    # Set y-axis range to better display the predictions
    min_pred = min(predictions)
    max_pred = max(predictions)
    padding = (max_pred - min_pred) * 0.1  # 10% padding
    fig.update_yaxes(range=[min_pred - padding, max_pred + padding])
    
    # Create results content
    results = html.Div([
        # Chart
        dcc.Graph(
            figure=fig,
            style={'height': '400px'}
        ),
        
        # Summary statistics
        html.Div([
            html.H5("Statistical Summary"),
            html.Div([
                html.Div([
                    html.H6("Initial Rating"),
                    html.P(f"{predictions[0]:.2f}", style={'fontSize': '24px', 'fontWeight': 'bold'})
                ], style={'textAlign': 'center', 'padding': '10px', 'backgroundColor': '#f8f9fa', 'borderRadius': '5px'}, className='four columns'),
                
                html.Div([
                    html.H6("Peak Rating"),
                    html.P(f"{max_prediction:.2f}", style={'fontSize': '24px', 'fontWeight': 'bold', 'color': '#d9534f'})
                ], style={'textAlign': 'center', 'padding': '10px', 'backgroundColor': '#f8f9fa', 'borderRadius': '5px'}, className='four columns'),
                
                html.Div([
                    html.H6("Final Rating"),
                    html.P([
                        f"{predictions[-1]:.2f} ",
                        html.Span(f"({predictions[-1] - predictions[0]:+.2f})", 
                                style={'color': 'green' if predictions[-1] >= predictions[0] else 'red', 'fontSize': '16px'})
                    ], style={'fontSize': '24px', 'fontWeight': 'bold'})
                ], style={'textAlign': 'center', 'padding': '10px', 'backgroundColor': '#f8f9fa', 'borderRadius': '5px'}, className='four columns'),
            ], className='row', style={'marginBottom': '20px'}),
        ]),

        # Interpretation
        html.Div([
            html.H5("Model Interpretation", style={'marginTop': '20px'}),
            html.P([
                f"Based on the provided data, this athlete's overall rating will start at {predictions[0]:.2f}, ",
                html.Br() if predictions[0] < max_prediction else "",
                f"reach a peak rating of {max_prediction:.2f} at age {max_age}, " if predictions[0] < max_prediction else "",
                html.Br() if predictions[-1] < max_prediction else "",
                f"then gradually decline to {predictions[-1]:.2f} with age." if predictions[-1] < max_prediction else f"and continue to improve to {predictions[-1]:.2f} with experience."
            ]),
            html.P("Note: This prediction assumes height and weight remain constant, only considering changes in age and years of experience over the next 10 years.", style={'fontStyle': 'italic'})
        ])
    ])
    
    return results, df

## Create panel

### Create FIFA panel

In [7]:
def create_fifa_panel():
    return html.Div([
        html.Div([
            html.H3("FIFA", style={'color': colors['fifa'], 'textAlign': 'center'}),
            html.P("FIFA players are evaluated based on their physical attributes and age.", 
                   style={'textAlign': 'center'}),
            
            # Add FIFA Model Features
            html.Div([
                html.H5("FIFA Model Features"),
                html.Ul([
                    html.Li("Height (cm)"),
                    html.Li("Weight (kg)"),
                    html.Li("Age"),
                    html.Li("BMI (automatically calculated)")
                ])
            ], style={'marginTop': '20px', 'marginBottom': '20px'}),
            
            # Prediction button
            html.Button('Predict FIFA 10-Year Career Changes', 
                        id='fifa-predict-button', 
                        style={'backgroundColor': colors['fifa'], 
                               'color': 'white', 
                               'padding': '10px', 
                               'marginTop': '10px',
                               'marginBottom': '10px',
                               'border': 'none',
                               'borderRadius': '5px',
                               'width': '100%',
                               'cursor': 'pointer'}),
            
            # Loading indicator and status
            dcc.Loading(
                id="fifa-loading",
                type="circle",
                color=colors['fifa'],
                children=[
                    html.Div(id='fifa-prediction-status', 
                             children="Ready to predict", 
                             style={'textAlign': 'center', 
                                    'marginTop': '10px', 
                                    'padding': '10px',
                                    'backgroundColor': '#f8f9fa',
                                    'borderRadius': '5px',
                                    'fontWeight': 'bold',
                                    'color': colors['text']})
                ]
            ),
        ], className='four columns', style={'padding': '15px', 'backgroundColor': '#FFFFFF', 'borderRadius': '5px', 'boxShadow': '0px 0px 5px lightgrey'}),
        
        # FIFA Results Panel
        html.Div([
            html.H3("FIFA Prediction Results", style={'textAlign': 'center', 'marginBottom': '20px'}),
            html.Div(id='fifa-prediction-results', children=[
                html.P("Click the 'Predict FIFA 20-Year Career Changes' button to see results.", style={'textAlign': 'center'})
            ])
        ], className='eight columns', style={'padding': '15px', 'backgroundColor': '#FFFFFF', 'borderRadius': '5px', 'boxShadow': '0px 0px 5px lightgrey'}),
        
        # Hidden div for FIFA data
        html.Div(id='fifa-intermediate-value', style={'display': 'none'}),
    ], className='row', style={'marginBottom': '20px'})

### Create NBA panel

In [8]:
def create_nba_panel():
    return html.Div([
        html.Div([
            html.H3("NBA", style={'color': colors['nba'], 'textAlign': 'center'}),
            html.P("NBA players are evaluated based on their position and experience.", 
                   style={'textAlign': 'center'}),
            
            # NBA-specific inputs
            html.Label("Years Professional"),
            dcc.Slider(
                id='nba-years-pro-slider', 
                min=0, 
                max=20, 
                step=1, 
                value=3, 
                marks={i: str(i) for i in range(0, 21, 5)},
                tooltip={"placement": "bottom", "always_visible": True}
            ),
            
            html.Label("Select Position", style={'marginTop': '15px'}),
            dcc.Dropdown(
                id='nba-position-dropdown',
                options=[
                    {'label': 'Point Guard (PG)', 'value': 1},
                    {'label': 'Shooting Guard (SG)', 'value': 2},
                    {'label': 'Small Forward (SF)', 'value': 3},
                    {'label': 'Power Forward (PF)', 'value': 4},
                    {'label': 'Center (C)', 'value': 5}
                ],
                value=1,
                clearable=False
            ),
            
            # Automatically calculated age entered pro
            html.Div([
                html.Label("Age Entered Pro:", style={'display': 'inline-block', 'marginRight': '10px', 'marginTop': '15px'}),
                html.Div(id='nba-enter-pro-display', style={'display': 'inline-block', 'fontWeight': 'bold', 'marginTop': '15px'})
            ]),
            
            # Add NBA Model Features
            html.Div([
                html.H5("NBA Model Features"),
                html.Ul([
                    html.Li("Height (cm)"),
                    html.Li("Weight (kg)"),
                    html.Li("Age"),
                    html.Li("Years Professional"),
                    html.Li("Position"),
                    html.Li("BMI (automatically calculated)"),
                    html.Li("Age Entered Pro (automatically calculated)")
                ])
            ], style={'marginTop': '20px', 'marginBottom': '20px'}),
            
            # Prediction button
            html.Button('Predict NBA 10-Year Career Changes', 
                        id='nba-predict-button', 
                        style={'backgroundColor': colors['nba'], 
                               'color': 'white', 
                               'padding': '10px', 
                               'marginTop': '10px',
                               'marginBottom': '10px',
                               'border': 'none',
                               'borderRadius': '5px',
                               'width': '100%',
                               'cursor': 'pointer'}),
            
            # Loading indicator and status
            dcc.Loading(
                id="nba-loading",
                type="circle",
                color=colors['nba'],
                children=[
                    html.Div(id='nba-prediction-status', 
                             children="Ready to predict", 
                             style={'textAlign': 'center', 
                                    'marginTop': '10px', 
                                    'padding': '10px',
                                    'backgroundColor': '#f8f9fa',
                                    'borderRadius': '5px',
                                    'fontWeight': 'bold',
                                    'color': colors['text']})
                ]
            ),
        ], className='four columns', style={'padding': '15px', 'backgroundColor': '#FFFFFF', 'borderRadius': '5px', 'boxShadow': '0px 0px 5px lightgrey'}),
        
        # NBA Results Panel
        html.Div([
            html.H3("NBA Prediction Results", style={'textAlign': 'center', 'marginBottom': '20px'}),
            html.Div(id='nba-prediction-results', children=[
                html.P("Click the 'Predict NBA 20-Year Career Changes' button to see results.", style={'textAlign': 'center'})
            ])
        ], className='eight columns', style={'padding': '15px', 'backgroundColor': '#FFFFFF', 'borderRadius': '5px', 'boxShadow': '0px 0px 5px lightgrey'}),
        
        # Hidden div for NBA data
        html.Div(id='nba-intermediate-value', style={'display': 'none'}),
    ], className='row', style={'marginBottom': '20px'})


### Create NFL panel

In [9]:
def create_nfl_panel():
    return html.Div([
        html.Div([
            html.H3("NFL", style={'color': colors['nfl'], 'textAlign': 'center'}),
            html.P("NFL players are evaluated based on their position and experience.", 
                   style={'textAlign': 'center'}),
            
            # NFL-specific inputs
            html.Label("Years Professional"),
            dcc.Slider(
                id='nfl-years-pro-slider', 
                min=0, 
                max=20, 
                step=1, 
                value=3, 
                marks={i: str(i) for i in range(0, 21, 5)},
                tooltip={"placement": "bottom", "always_visible": True}
            ),
            
            html.Label("Select Position", style={'marginTop': '15px'}),
            dcc.Dropdown(
                id='nfl-position-dropdown',
                options=[
                    {'label': 'Quarterback (QB)', 'value': 1},
                    {'label': 'Receiver', 'value': 2},
                    {'label': 'Defensive Line (DL)', 'value': 3},
                    {'label': 'Offensive Line (OL)', 'value': 4},
                    {'label': 'Defensive Back (DB)', 'value': 5},
                    {'label': 'Linebacker (LB)', 'value': 6},
                    {'label': 'Running Back (RB)', 'value': 7},
                    {'label': 'Specialist', 'value': 8}
                ],
                value=1,
                clearable=False
            ),
            
            # Automatically calculated age entered pro
            html.Div([
                html.Label("Age Entered Pro:", style={'display': 'inline-block', 'marginRight': '10px', 'marginTop': '15px'}),
                html.Div(id='nfl-enter-pro-display', style={'display': 'inline-block', 'fontWeight': 'bold', 'marginTop': '15px'})
            ]),
            
            # Add NFL Model Features
            html.Div([
                html.H5("NFL Model Features"),
                html.Ul([
                    html.Li("Height (cm)"),
                    html.Li("Weight (kg)"),
                    html.Li("Age"),
                    html.Li("Years Professional"),
                    html.Li("Position"),
                    html.Li("BMI (automatically calculated)"),
                    html.Li("Age Entered Pro (automatically calculated)")
                ])
            ], style={'marginTop': '20px', 'marginBottom': '20px'}),
            
            # Prediction button
            html.Button('Predict NFL 10-Year Career Changes', 
                        id='nfl-predict-button', 
                        style={'backgroundColor': colors['nfl'], 
                               'color': 'white', 
                               'padding': '10px', 
                               'marginTop': '10px',
                               'marginBottom': '10px',
                               'border': 'none',
                               'borderRadius': '5px',
                               'width': '100%',
                               'cursor': 'pointer'}),
            
            # Loading indicator and status
            dcc.Loading(
                id="nfl-loading",
                type="circle",
                color=colors['nfl'],
                children=[
                    html.Div(id='nfl-prediction-status', 
                             children="Ready to predict", 
                             style={'textAlign': 'center', 
                                    'marginTop': '10px', 
                                    'padding': '10px',
                                    'backgroundColor': '#f8f9fa',
                                    'borderRadius': '5px',
                                    'fontWeight': 'bold',
                                    'color': colors['text']})
                ]
            ),
        ], className='four columns', style={'padding': '15px', 'backgroundColor': '#FFFFFF', 'borderRadius': '5px', 'boxShadow': '0px 0px 5px lightgrey'}),
        
        # NFL Results Panel
        html.Div([
            html.H3("NFL Prediction Results", style={'textAlign': 'center', 'marginBottom': '20px'}),
            html.Div(id='nfl-prediction-results', children=[
                html.P("Click the 'Predict NFL 20-Year Career Changes' button to see results.", style={'textAlign': 'center'})
            ])
        ], className='eight columns', style={'padding': '15px', 'backgroundColor': '#FFFFFF', 'borderRadius': '5px', 'boxShadow': '0px 0px 5px lightgrey'}),
        
        # Hidden div for NFL data
        html.Div(id='nfl-intermediate-value', style={'display': 'none'}),
    ], className='row', style={'marginBottom': '20px'})

## Callback

## Initialize Dash application and color scheme


In [10]:
app = dash.Dash(__name__, suppress_callback_exceptions=True)
app.title = "Athlete Career Prediction Dashboard"

# Define color scheme
colors = {
    'background': '#F9F9F9',
    'text': '#333333',
    'accent': '#1E88E5',
    'highlight': '#FFC107',
    'chart': '#2196F3',
    'fifa': '#EA2027',
    'nba': '#009432',
    'nfl': '#0652DD'
}

header = html.Div([
    html.H1("Athlete Career Prediction Dashboard", style={'textAlign': 'center', 'color': colors['text']}),
    html.Hr(),
    html.H4("Enter athlete data to predict overall rating changes over the next 10 years", 
            style={'textAlign': 'center', 'color': colors['text']}),
    html.Hr(),
])

footer = html.Div([
    html.Hr(),
    html.H4("About This Dashboard"),
    html.P("This tool uses machine learning models to predict athletes' future overall ratings based on physical data, age, and position."),
    html.P("Note: All predictions are for reference only. Actual results may vary due to injuries, training, and other factors."),
], style={'marginTop': '30px', 'paddingTop': '20px', 'borderTop': '1px solid #eee'})

# Define app layout
app.layout = html.Div([
    header,
    
    # Shared metrics at the top
    create_shared_metrics(),
    
    # Separate panels for each sport
    create_fifa_panel(),
    create_nba_panel(),
    create_nfl_panel(),
    
    footer,
], style={'padding': '20px', 'backgroundColor': colors['background']})

In [11]:
@app.callback(
    Output('bmi-display', 'children'),
    [Input('height-slider', 'value'),
     Input('weight-slider', 'value')]
)
def calculate_bmi(height, weight):
    if height and weight:
        bmi = weight / ((height/100) ** 2)
        return f"{bmi:.2f}"
    return "N/A"

# Callback to automatically calculate NBA age entered pro league
@app.callback(
    Output('nba-enter-pro-display', 'children'),
    [Input('age-slider', 'value'),
     Input('nba-years-pro-slider', 'value')]
)
def calculate_nba_enter_pro(age, years_pro):
    if age and years_pro is not None:
        enter_pro = age - years_pro
        return f"{enter_pro}"
    return "N/A"

# Callback to automatically calculate NFL age entered pro league
@app.callback(
    Output('nfl-enter-pro-display', 'children'),
    [Input('age-slider', 'value'),
     Input('nfl-years-pro-slider', 'value')]
)
def calculate_nfl_enter_pro(age, years_pro):
    if age and years_pro is not None:
        enter_pro = age - years_pro
        return f"{enter_pro}"
    return "N/A"

## FIFA prediction

In [12]:
@app.callback(
    [Output('fifa-intermediate-value', 'children'),
     Output('fifa-prediction-status', 'children'),
     Output('fifa-prediction-status', 'style')],
    [Input('fifa-predict-button', 'n_clicks')],
    [State('height-slider', 'value'),
     State('weight-slider', 'value'),
     State('age-slider', 'value'),
     State('bmi-display', 'children')]
)
def update_fifa_prediction(n_clicks, height, weight, age, bmi):
    if not n_clicks:
        # No clicks yet
        status_style = {'textAlign': 'center', 
                        'marginTop': '10px', 
                        'padding': '10px',
                        'backgroundColor': '#f8f9fa',
                        'borderRadius': '5px',
                        'fontWeight': 'bold',
                        'color': colors['text']}
        return None, "Ready to predict", status_style
    
    try:
        # Convert BMI to float
        try:
            bmi_value = float(bmi)
        except:
            bmi_value = weight / ((height/100) ** 2)
        
        # Start prediction process
        status_style = {'textAlign': 'center', 
                        'marginTop': '10px', 
                        'padding': '10px',
                        'backgroundColor': '#fff3cd',  # Yellow background
                        'borderRadius': '5px',
                        'fontWeight': 'bold',
                        'color': colors['text']}
        
        # Collect input data
        input_data = get_input_data(
            'FIFA',
            height,
            weight,
            age,
            bmi_value
        )
        
        # Generate predictions
        predictions, ages, features = predict_future_ratings('FIFA', input_data)
        
        # Create results
        results, df = create_prediction_results(predictions, ages, colors['fifa'])
        
        # Store results
        output_data = {
            'predictions': predictions,
            'ages': ages,
            'df': df.to_dict('records')
        }
        
        # Prediction complete
        status_style = {'textAlign': 'center', 
                        'marginTop': '10px', 
                        'padding': '10px',
                        'backgroundColor': '#d4edda',  # Green background
                        'borderRadius': '5px',
                        'fontWeight': 'bold',
                        'color': colors['text']}
        
        return json.dumps(output_data), "Prediction complete!", status_style
    
    except Exception as e:
        # Error during prediction
        print(f"FIFA prediction error: {e}")
        status_style = {'textAlign': 'center', 
                        'marginTop': '10px', 
                        'padding': '10px',
                        'backgroundColor': '#f8d7da',  # Red background
                        'borderRadius': '5px',
                        'fontWeight': 'bold',
                        'color': colors['text']}
        return None, f"Error: {str(e)}", status_style

@app.callback(
    Output('fifa-prediction-results', 'children'),
    [Input('fifa-intermediate-value', 'children')]
)
def update_fifa_results(intermediate_value):
    if not intermediate_value:
        return html.P("Click the 'Predict FIFA 10-Year Career Changes' button to see results.", style={'textAlign': 'center'})
    
    try:
        # Parse the intermediate value
        data = json.loads(intermediate_value)
        predictions = data.get('predictions', [])
        ages = data.get('ages', [])
        
        # Create the results
        results, _ = create_prediction_results(predictions, ages, colors['fifa'])
        return results
    
    except Exception as e:
        print(f"Error updating FIFA results: {e}")
        return html.Div([
            html.H4("Error Processing Results", style={'color': 'red'}),
            html.P(f"An error occurred: {str(e)}"),
            html.P("Please try again with different parameters.")
        ])



## NBA PREDICTION

In [13]:
@app.callback(
    [Output('nba-intermediate-value', 'children'),
     Output('nba-prediction-status', 'children'),
     Output('nba-prediction-status', 'style')],
    [Input('nba-predict-button', 'n_clicks')],
    [State('height-slider', 'value'),
     State('weight-slider', 'value'),
     State('age-slider', 'value'),
     State('bmi-display', 'children'),
     State('nba-position-dropdown', 'value'),
     State('nba-years-pro-slider', 'value'),
     State('nba-enter-pro-display', 'children')]
)
def update_nba_prediction(n_clicks, height, weight, age, bmi, position, years_pro, enter_pro):
    if not n_clicks:
        # No clicks yet
        status_style = {'textAlign': 'center', 
                        'marginTop': '10px', 
                        'padding': '10px',
                        'backgroundColor': '#f8f9fa',
                        'borderRadius': '5px',
                        'fontWeight': 'bold',
                        'color': colors['text']}
        return None, "Ready to predict", status_style
    
    try:
        # Convert BMI to float
        try:
            bmi_value = float(bmi)
        except:
            bmi_value = weight / ((height/100) ** 2)
            
        # Convert enter_pro to int
        try:
            if enter_pro is not None and enter_pro != "N/A":
                enter_pro_value = int(enter_pro)
            else:
                enter_pro_value = age - years_pro if years_pro is not None else age
        except:
            enter_pro_value = age - years_pro
        
        # Start prediction process
        status_style = {'textAlign': 'center', 
                        'marginTop': '10px', 
                        'padding': '10px',
                        'backgroundColor': '#fff3cd',  # Yellow background
                        'borderRadius': '5px',
                        'fontWeight': 'bold',
                        'color': colors['text']}
        
        # Collect input data
        input_data = get_input_data(
            'NBA',
            height,
            weight,
            age,
            bmi_value,
            position,
            years_pro,
            enter_pro_value
        )
        
        # Generate predictions
        predictions, ages, features = predict_future_ratings('NBA', input_data)
        
        # Create results
        results, df = create_prediction_results(predictions, ages, colors['nba'])
        
        # Store results
        output_data = {
            'predictions': predictions,
            'ages': ages,
            'df': df.to_dict('records')
        }
        
        # Prediction complete
        status_style = {'textAlign': 'center', 
                        'marginTop': '10px', 
                        'padding': '10px',
                        'backgroundColor': '#d4edda',  # Green background
                        'borderRadius': '5px',
                        'fontWeight': 'bold',
                        'color': colors['text']}
        
        return json.dumps(output_data), "Prediction complete!", status_style
    
    except Exception as e:
        # Error during prediction
        print(f"NBA prediction error: {e}")
        status_style = {'textAlign': 'center', 
                        'marginTop': '10px', 
                        'padding': '10px',
                        'backgroundColor': '#f8d7da',  # Red background
                        'borderRadius': '5px',
                        'fontWeight': 'bold',
                        'color': colors['text']}
        return None, f"Error: {str(e)}", status_style
    
@app.callback(
    Output('nba-prediction-results', 'children'),
    [Input('nba-intermediate-value', 'children')]
)
def update_nba_results(intermediate_value):
    if not intermediate_value:
        return html.P("Click the 'Predict NBA 10-Year Career Changes' button to see results.", style={'textAlign': 'center'})
    
    try:
        # Parse the intermediate value
        data = json.loads(intermediate_value)
        predictions = data.get('predictions', [])
        ages = data.get('ages', [])
        
        # Create the results
        results, _ = create_prediction_results(predictions, ages, colors['nba'])
        return results
    
    except Exception as e:
        print(f"Error updating NBA results: {e}")
        return html.Div([
            html.H4("Error Processing Results", style={'color': 'red'}),
            html.P(f"An error occurred: {str(e)}"),
            html.P("Please try again with different parameters.")
        ])

## NFL PREDICTION

In [14]:
@app.callback(
    [Output('nfl-intermediate-value', 'children'),
     Output('nfl-prediction-status', 'children'),
     Output('nfl-prediction-status', 'style')],
    [Input('nfl-predict-button', 'n_clicks')],
    [State('height-slider', 'value'),
     State('weight-slider', 'value'),
     State('age-slider', 'value'),
     State('bmi-display', 'children'),
     State('nfl-position-dropdown', 'value'),
     State('nfl-years-pro-slider', 'value'),
     State('nfl-enter-pro-display', 'children')]
)
def update_nfl_prediction(n_clicks, height, weight, age, bmi, position, years_pro, enter_pro):
    if not n_clicks:
        # No clicks yet
        status_style = {'textAlign': 'center', 
                        'marginTop': '10px', 
                        'padding': '10px',
                        'backgroundColor': '#f8f9fa',
                        'borderRadius': '5px',
                        'fontWeight': 'bold',
                        'color': colors['text']}
        return None, "Ready to predict", status_style
    
    try:
        # Convert BMI to float
        try:
            bmi_value = float(bmi)
        except:
            bmi_value = weight / ((height/100) ** 2)
            
        # Convert enter_pro to int
        try:
            if enter_pro is not None and enter_pro != "N/A":
                enter_pro_value = int(enter_pro)
            else:
                enter_pro_value = age - years_pro if years_pro is not None else age
        except:
            enter_pro_value = age - years_pro
        
        # Start prediction process
        status_style = {'textAlign': 'center', 
                        'marginTop': '10px', 
                        'padding': '10px',
                        'backgroundColor': '#fff3cd',  # Yellow background
                        'borderRadius': '5px',
                        'fontWeight': 'bold',
                        'color': colors['text']}
        
        # Collect input data
        input_data = get_input_data(
            'NFL',
            height,
            weight,
            age,
            bmi_value,
            position,
            years_pro,
            enter_pro_value
        )
        
        # Generate predictions
        predictions, ages, features = predict_future_ratings('NFL', input_data)
        
        # Create results
        results, df = create_prediction_results(predictions, ages, colors['nfl'])
        
        # Store results
        output_data = {
            'predictions': predictions,
            'ages': ages,
            'df': df.to_dict('records')
        }
        
        # Prediction complete
        status_style = {'textAlign': 'center', 
                        'marginTop': '10px', 
                        'padding': '10px',
                        'backgroundColor': '#d4edda',  # Green background
                        'borderRadius': '5px',
                        'fontWeight': 'bold',
                        'color': colors['text']}
        
        return json.dumps(output_data), "Prediction complete!", status_style
    
    except Exception as e:
        # Error during prediction
        print(f"NFL prediction error: {e}")
        status_style = {'textAlign': 'center', 
                        'marginTop': '10px', 
                        'padding': '10px',
                        'backgroundColor': '#f8d7da',  # Red background
                        'borderRadius': '5px',
                        'fontWeight': 'bold',
                        'color': colors['text']}
        return None, f"Error: {str(e)}", status_style
@app.callback(
    Output('nfl-prediction-results', 'children'),
    [Input('nfl-intermediate-value', 'children')]
)
def update_nfl_results(intermediate_value):
    if not intermediate_value:
        return html.P("Click the 'Predict NFL 10-Year Career Changes' button to see results.", style={'textAlign': 'center'})
    
    try:
        # Parse the intermediate value
        data = json.loads(intermediate_value)
        predictions = data.get('predictions', [])
        ages = data.get('ages', [])
        
        # Create the results
        results, _ = create_prediction_results(predictions, ages, colors['nfl'])
        return results
    
    except Exception as e:
        print(f"Error updating NFL results: {e}")
        return html.Div([
            html.H4("Error Processing Results", style={'color': 'red'}),
            html.P(f"An error occurred: {str(e)}"),
            html.P("Please try again with different parameters.")
        ])

## Dash board

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)



c:\Users\Champion\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names

c:\Users\Champion\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names

c:\Users\Champion\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names

c:\Users\Champion\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names

c:\Users\Champion\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names

c:\Users\Champion\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid fe